In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [2]:
from proj1_helpers import load_csv_data
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, x, ids = load_csv_data(DATA_TRAIN_PATH)
# TODO: convert labels to -1,1 ?

## Note: This is the raw dataset, you can also work with your modified features if you prefer

In [3]:
y = np.array([y]).T
print(y.shape)
print(x.shape)
xt = x.T
print(xt.shape)

(250000, 1)
(250000, 30)
(30, 250000)


In [4]:
def calculate_cost(y, x, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer."""
    # Here x is the full dataset matrix, and y are the corresponding +1 or -1 labels
    # ***************************************************
    zeros = np.zeros((x.shape[0],1))
    hinge_loss = np.ones((x.shape[0],1)) - y * np.dot(x, w)
    #print(hinge_loss.shape)
    hinge_loss = np.maximum(zeros, hinge_loss)
    #print(hinge_loss.shape)
    hinge_penalty = np.ones((y.shape[0],1)) * (lambda_/2) * np.linalg.norm(w[0])**2
    #print(hinge_penalty.shape)
    loss = np.sum(hinge_loss + hinge_penalty)
#     print(loss)
    # ***************************************************
    return loss
ww = np.zeros((x.shape[1],1))
calculate_cost(y, x, ww, 0.1)

250000.0

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [5]:
print(x[1].shape)
print(y[1].shape)

(30,)
(1,)


In [6]:
def calculate_gradient(y, x, w, lambda_, n):
    """compute the stochastic gradient of loss plus regularizer."""
    # Here x is one datapoint, and y is the corresponding +1 or -1 label
    # 
    # ***************************************************
    yy = np.array([y[n]])
    xx = np.array([x[n]])
#     print(yy.shape)
#     print(xx.shape)

    differentiable = 1 - yy * np.dot(xx, w)
    if differentiable > 0:
        hinge_loss_gd = -yy * xx
    else:
        hinge_loss_gd = 0 * xx
    gradient = hinge_loss_gd.T + lambda_ * w
    # ***************************************************
#     hinge_loss_gd = 1 - yy * np.dot(xx, w)
#     if hinge_loss_gd < 1:
#         Ix = 1
#     else:
#         Ix = 0
#     #hinge_loss_gd = np.maximum(zeros, hinge_loss_gd)
#     gradient = w * Ix * (np.dot(xx, w)-yy) + lambda_ * w
    # ***************************************************
    #Classical MSE SGD
#     e = np.dot(xx, w) - yy
#     gradient = np.dot(xx.T, e) + (lambda_/2) * (w**2)
    # ***************************************************
    # Be careful about the constant N(size) term! The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    return gradient
calculate_gradient(y, x, np.zeros((x.shape[1],1)), 0.1, 1)

array([[  1.60937000e+02],
       [  6.87680000e+01],
       [  1.03235000e+02],
       [  4.81460000e+01],
       [ -9.99000000e+02],
       [ -9.99000000e+02],
       [ -9.99000000e+02],
       [  3.47300000e+00],
       [  2.07800000e+00],
       [  1.25157000e+02],
       [  8.79000000e-01],
       [  1.41400000e+00],
       [ -9.99000000e+02],
       [  4.20140000e+01],
       [  2.03900000e+00],
       [ -3.01100000e+00],
       [  3.69180000e+01],
       [  5.01000000e-01],
       [  1.03000000e-01],
       [  4.47040000e+01],
       [ -1.91600000e+00],
       [  1.64546000e+02],
       [  1.00000000e+00],
       [  4.62260000e+01],
       [  7.25000000e-01],
       [  1.15800000e+00],
       [ -9.99000000e+02],
       [ -9.99000000e+02],
       [ -9.99000000e+02],
       [  4.62260000e+01]])

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [ ]:
def sgd_for_svm_demo(y, x):
    # ***************************************************
    # INSERT YOUR CODE HERE
    # classify the data by SGD for SVM: TODO
    # ***************************************************
    max_iter = 1000
    gamma = 0.001
    lambda_ = 1.0 / y.shape[0]  # or set to a different value, try cross-validation!
    
    w = np.zeros((x.shape[1], 1))
    
    for iter in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = int(np.random.uniform(low=0, high=x.shape[0]))
        grad = calculate_gradient(y, x, w, lambda_, n)
        w = w - (gamma * grad)
        loss = calculate_cost(y, x, w, lambda_)
        #print(loss)
        
        if iter % 100 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
    
    print("Objective = {l}".format(l=calculate_cost(y, x, w, lambda_)))

sgd_for_svm_demo(y, x)

Current iteration=0, the loss=382326982.99186164
Current iteration=100, the loss=109074954.78850296
Current iteration=200, the loss=343502163.71438706
Current iteration=300, the loss=263845570.77143487
Current iteration=400, the loss=1193988179.7111957
Current iteration=500, the loss=365468393.1370788
Current iteration=600, the loss=223474406.450875
Current iteration=700, the loss=400997183.0576159
Current iteration=800, the loss=274590141.5101561
Current iteration=900, the loss=109193261.03351654
Objective = 187866347.29539007


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [ ]:
Y = np.diag(y[:,0])
print(np.dot(x, x.T).shape)
# print(np.dot(x.T, Y).shape)
# Q = np.dot(Y, np.dot(x , np.dot(x.T, Y)))
# print(Q.shape)

In [ ]:
def calculate_coordinate_update(y, x, lambda_, n, w):
    # Here x is one datapoint, and y is the corresponding +1 or -1 label
    # ***************************************************
    Y = np.diag(y[:,0])
    Q = np.dot(Y, np.dot(x , np.dot(x.T, Y)))
    a_t = np.array([x[n]])
    alpha = a_t - 1/(2*lambda_) * (np.dot(a_t, np.dot(Q, a_t.T)))
    # ***************************************************
    return alpha
calculate_coordinate_update(y, x, 0.1, 0, np.zeros((x.shape[1],1)))

In [ ]:
def coordinate_descent_for_svm_demo(y, x):
    # ***************************************************
    # INSERT YOUR CODE HERE
    # classify the data by SGD for SVM: TODO
    # ***************************************************
    max_iter = 10000
    gamma = 0.001
    lambda_ = 1.0 / y.shape[0]

    w = np.zeros((x.shape[1], 1))
    
    for iter in range(max_iter):
        # n = uniformly random data point from x
        n = int(np.random.uniform(low=0, high=x.shape[0]))
        loss = calculate_cost(y, x, w, lamda_)
        alpha = calculate_coordinate_update(y, x, lambda_, n, w)
        
        w = 1/lambda_ * np.dot(x.T, np.dot(Y, alpha)) # XYa
        
        if iter % 1000 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
    
    print("Primal objective = {l}".format(l=calculate_cost(y, x, w, lambda_)))

coordinate_descent_for_svm_demo(y, x)